## Preparation

In [ ]:
%%capture
!pip install pytorch-lightning
# !pip install torchmetrics
# !pip install tqdm

In [ ]:
# Setting up google drive 
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/MyDrive/Colab Notebooks/code'

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/code


In [ ]:
import pandas as pd
import numpy as np
import os.path
from IPython.utils import io

import torch
from torch import nn
from torch.nn.functional import interpolate
from torch.utils.data import Dataset, DataLoader
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
from torchmetrics.functional import accuracy

import seaborn as sns
from pylab import rcParams
from tqdm.auto import tqdm
import my_utils as mu
import project_utils as pu

from project_Prepro import preprocess_XY, preprocess_Y, preprocess_Y2, Split_Interpolate_RNN, Split_Pad
from project_Data import PhyAAtDataModule
from project_RNN import RNNmodel

## Within-Subject Experiment

In [ ]:
# task label - task mapping
task_label = {
    1:'Correctness_label',
    2:'SNRdB_label',
    3:'Semanticity_label'
    }

# dataframe to save all results
ls = ['S{}'.format(i) for i in range(1,26)]
col = ['Task{}'.format(i) for i in range(1,4)]
result = pd.DataFrame(index = ls, columns = col)

BATCH_SIZE = 20
N_EPOCHS = 100

In [ ]:
# loop over all subjects
# for all subjects change 2 to 26
for subject_No in range(1,2):
    print("Subject No." + str(subject_No) + ":")
    path = 'X{}_trial.csv'.format(subject_No)
    Y = pd.read_csv('S{0}_Textscore.csv'.format(subject_No))
    
    # 1 Data Input and process
    if os.path.exists(path):
      X = pd.read_csv(path)
      Y = preprocess_Y(subject_No)
    else:
      X, Y = preprocess_XY(subject_No)
      X.to_csv('X{}_trial.csv'.format(subject_No),index = False)

    FEATURE_COLUMNS = X.columns[1:15].tolist()

    # loop over all tasks
    # for all tasks change 2 to 4
    for target_task in range(1,2):
        print("Task:",target_task)
        target_label = task_label[target_task]

        # 2 Interpolate, Split into trials, and split into train, valid, and test sequences
        X_trial = Split_Pad(X,Y,target_label)
        
        train_sequences, test_sequences = train_test_split(X_trial, test_size = 0.25)
        valid_sequences, test_sequences = train_test_split(test_sequences, test_size = 0.4) # train:valid:test = 75:15:10

        # 3 Dataset DataModuel
        data_module = PhyAAtDataModule(train_sequences, valid_sequences, test_sequences, BATCH_SIZE)
        
        # 4 Net and Model
        model = RNNmodel(
            num_features = len(FEATURE_COLUMNS),
            num_classes = len(Y[task_label[target_task]].unique())
        )
        
        # 5 Train
        earlystopping_callback = EarlyStopping(
            monitor = 'val_accuracy',
            mode = 'max',
            patience = 10
        )
        checkpoint_callback = ModelCheckpoint(
            # dirpath = 'lightning_logs/checkpoints',
            # filename = '{epoch:02d}-{val_accuracy:.2f}-{val_loss:.2f}' ,
            save_top_k = 1,
            verbose = True,
            monitor = 'val_accuracy',
            mode = "max"
        )
        callback_list = [checkpoint_callback, earlystopping_callback]
        # logger = TensorBoardLogger("lightning_logs", name = task_label[target_task])
        # logger = TensorBoardLogger("lightning_logs", name = "Correctness")

        trainer = pl.Trainer(
        #     logger=logger,
            callbacks=callback_list,
            max_epochs=N_EPOCHS,
            accelerator = 'gpu',
            devices = 1
            # ,auto_lr_find=True
        )
        
        trainer.fit(model, data_module)

        # 6 Test and result
        test_result = trainer.test(model, dataloaders = data_module.test_dataloader())
        result.at["S{}".format(subject_No), "Task{}".format(target_task)] = round(test_result[0]['test_accuracy'],2)

Subject No.25:
Task: 1


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | model     | RNNnet           | 148 K 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
148 K     Trainable params
0         Non-trainable params
148 K     Total params
0.595     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/content/gdrive/MyDrive/Colab Notebooks/code/project_Data.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feature = torch.tensor(feature).clone().detach()
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1898: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 10, global step 22: 'val_accuracy' was not in top 1
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

/content/gdrive/MyDrive/Colab Notebooks/code/project_Data.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  feature = torch.tensor(feature).clone().detach()


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy          0.800000011920929
        test_loss           0.6815471649169922
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
result

,Task1,Task2,Task3
S1,0.47,0.4,0.47
S2,0.47,0.4,0.6
S3,0.47,0.6,0.67
S4,0.47,0.53,0.47
S5,0.6,0.6,0.6
S6,0.47,0.47,0.27
S7,0.47,0.53,0.53
S8,0.53,0.53,0.73
S9,0.47,0.53,0.33
S10,0.2,0.33,0.47


In [ ]:
result.at['Avg_Acc'] = round(result.mean(axis = 0, skipna = True),2)
chance_acc = [0.1, 0.17, 0.5]
result.at['Chance_Acc'] = chance_acc
result = result.drop(index = 'S24')
result.to_csv('result_rnn_pad.csv')
result.plot.bar(subplots = True, figsize = (20, 6), ylabel = 'Accuracy', rot = 45)